In [ ]:
%cd lit-gpt

In [ ]:
# !wget https://gist.githubusercontent.com/aniketmaurya/b8e5bd3f1594bd31ed34375ed916f075/raw/b831e5ab054ac6c94a4a409d32be14b6fdcad82a/databricks-dolly-15k.csv
# !mv databricks-dolly-15k.csv /data/aniket/datasets/databricks-dolly-15k.csv

In [ ]:
import pandas as pd

df = pd.read_csv("/data/aniket/datasets/databricks-dolly-15k.csv")

In [ ]:
df.head()

In [ ]:
COLUMNS = ["instruction", "input", "output"]
df[COLUMNS].to_csv("databricks-dolly-15k.csv", index=False)

In [ ]:
!python scripts/prepare_csv.py \
    --csv_path "databricks-dolly-15k.csv" \
    --checkpoint_dir "/data/aniket/Llama-2-7b-hf" \
    --destination_path "data/dolly" \
    --max_seq_length 512

In [ ]:
!python finetune/lora.py \
    --checkpoint_dir "/data/aniket/Llama-2-7b-hf" \
    --data_dir "data/dolly" \
    --out_dir "out/lora/dolly"

In [ ]:
!python finetune/lora.py \
    --checkpoint_dir "/data/aniket/Llama-2-7b-hf" \
    --data_dir "data/dolly" \
    --out_dir "out/lora/dolly" \
    --precision bf16-true

In [ ]:
!python finetune/lora.py \
    --checkpoint_dir "/data/aniket/Llama-2-7b-hf" \
    --data_dir "data/dolly" \
    --out_dir "out/lora/dolly" \
    --precision bf16-true \
    --quantize bnb.fp4 

In [ ]:
!python scripts/merge_lora.py \
    --checkpoint_dir "/data/aniket/Llama-2-7b-hf" \
    --lora_path "out/dolly/Llama-2-7b-hf/lit_model_lora_finetuned.pth" \
    --out_dir "out/dolly/Llama-2-7b-hf/"


!python eval/lm_eval_harness.py \
    --checkpoint_dir "/data/aniket/Llama-2-7b-hf" \
    --eval_tasks "[truthfulqa_mc]" \
    --precision "bf16-true" \
    --batch_size 4 \
    --save_filepath "results.json"